# Exp Q4: Cosine Temporal Evolution

## 目的
cos(g_struct, g_value)の時間発展を詳細にログし、
「critical」が cosの符号反転点／ゼロ交差と対応するかを検証。

## 実験設計
- **代表条件のみ（不要に増やさない）**
- **η=0.4**: λ = [0.35 (ordered), 0.48 (critical), 0.60 (collapse)]
- **η=0.8**: λ = [0.35, 0.44, 0.55]
- **Seeds**: 5
- **Logging**: 各epoch の cos, ||g_mix||, test_error
- **Total**: 2 × 3 × 5 = **30 runs**

## 推定時間
~30 runs × 10 min ≈ **5 時間**（詳細ログのため若干長め）

In [ ]:
# ===== セットアップ =====
from google.colab import drive
drive.mount('/content/drive')

import os
import glob
from datetime import datetime

EXP_NAME = 'exp_Q4_cos_temporal'
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'

existing = glob.glob(f'{BASE_DIR}/{EXP_NAME}_*')
if existing:
    SAVE_DIR = sorted(existing)[-1]
    print(f'🔄 Resuming from: {SAVE_DIR}')
else:
    TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
    SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f'🆕 New experiment: {SAVE_DIR}')

os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)
print(f'Save directory: {SAVE_DIR}')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np
import json
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name()}')

In [ ]:
# ===== ResNet18 for CIFAR-10 =====
def get_resnet18():
    model = resnet18(weights=None, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    return model

# ===== IndexedDataset =====
class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

In [ ]:
# ===== 実験パラメータ =====
BATCH_SIZE = 256
NUM_WORKERS = 4
EPOCHS = 100
LR = 0.1
K = 16

# 代表条件
NOISE_LAMBDA_MAP = {
    0.4: {
        0.35: 'ordered',
        0.48: 'critical',
        0.60: 'collapse'
    },
    0.8: {
        0.35: 'ordered',
        0.44: 'critical',
        0.55: 'collapse'
    }
}

SEEDS = list(range(5))

experiments = []
for eta, lambdas in NOISE_LAMBDA_MAP.items():
    for lam, phase_label in lambdas.items():
        for seed in SEEDS:
            experiments.append({
                'noise_rate': eta,
                'lambda': lam,
                'phase_label': phase_label,
                'seed': seed
            })

print(f'Noise-Lambda map: {NOISE_LAMBDA_MAP}')
print(f'Total experiments: {len(experiments)}')
print(f'Estimated time: {len(experiments) * 10 / 60:.1f} hours')

# Save config
config = {
    'experiment': EXP_NAME,
    'description': 'Cosine temporal evolution: detailed logging of cos dynamics',
    'parameters': {
        'noise_lambda_map': {str(k): v for k, v in NOISE_LAMBDA_MAP.items()},
        'seeds': SEEDS,
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'lr': LR,
        'K': K
    },
    'total_runs': len(experiments)
}
with open(f'{SAVE_DIR}/{EXP_NAME}_config.json', 'w') as f:
    json.dump(config, f, indent=2)

In [ ]:
# ===== ユーティリティ関数 =====
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

def load_cifar10():
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
    trainset = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=transform_train)
    testset = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=transform_test)
    return trainset, testset

def inject_label_noise(labels, noise_rate, seed):
    np.random.seed(seed)
    labels = np.array(labels)
    n_samples = len(labels)
    n_noisy = int(noise_rate * n_samples)
    noisy_idx = np.random.choice(n_samples, n_noisy, replace=False)
    noisy_labels = labels.copy()
    for idx in noisy_idx:
        choices = [l for l in range(10) if l != labels[idx]]
        noisy_labels[idx] = np.random.choice(choices)
    return noisy_labels

def get_data_loaders(trainset, testset):
    train_loader = DataLoader(IndexedDataset(trainset), batch_size=BATCH_SIZE,
                              shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE,
                             shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
    return train_loader, test_loader

def evaluate(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total

In [ ]:
# ===== Dual-Gradient Learning with Detailed Logging =====
def train_dual_gradient_detailed(model, train_loader, test_loader, clean_labels, noisy_labels, lam):
    optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 75], gamma=0.1)
    criterion = nn.CrossEntropyLoss()
    
    clean_labels_tensor = torch.tensor(clean_labels, device=device)
    noisy_labels_tensor = torch.tensor(noisy_labels, device=device)
    
    cached_g_value = None
    global_step = 0
    
    # 詳細な履歴を記録
    detailed_history = {
        'epoch': [],
        'test_error': [],
        'train_loss': [],
        # Cosine metrics
        'cos_struct_value_mean': [],
        'cos_struct_value_std': [],
        'cos_struct_value_min': [],
        'cos_struct_value_max': [],
        # Gradient norms
        'g_struct_norm_mean': [],
        'g_value_norm_mean': [],
        'g_mix_norm_mean': [],
        # Additional: cos with clean gradient
        'cos_mix_clean_mean': []
    }
    
    for epoch in range(EPOCHS):
        model.train()
        epoch_cos_sv = []  # struct-value
        epoch_cos_mc = []  # mix-clean
        epoch_g_struct_norm = []
        epoch_g_value_norm = []
        epoch_g_mix_norm = []
        epoch_loss = []
        
        for inputs, _, indices in train_loader:
            inputs = inputs.to(device, non_blocking=True)
            indices = indices.to(device, non_blocking=True)
            batch_noisy = noisy_labels_tensor[indices]
            batch_clean = clean_labels_tensor[indices]
            
            # Structure gradient
            optimizer.zero_grad()
            outputs = model(inputs)
            loss_struct = criterion(outputs, batch_noisy)
            loss_struct.backward(retain_graph=True)
            g_struct = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            # Value gradient (clean)
            if global_step % K == 0 or cached_g_value is None:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss_value = criterion(outputs, batch_clean)
                loss_value.backward()
                cached_g_value = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            # Compute normalized gradients
            g_struct_norm_val = g_struct.norm().item()
            g_value_norm_val = cached_g_value.norm().item()
            
            g_struct_normalized = g_struct / (g_struct_norm_val + 1e-12)
            g_value_normalized = cached_g_value / (g_value_norm_val + 1e-12)
            
            # Cosine: struct vs value
            cos_sv = (g_struct_normalized @ g_value_normalized).item()
            epoch_cos_sv.append(cos_sv)
            
            # Mix
            g_mix = (1 - lam) * g_struct_normalized + lam * g_value_normalized
            g_mix_norm_val = g_mix.norm().item()
            
            # Cosine: mix vs clean (value)
            g_mix_normalized = g_mix / (g_mix_norm_val + 1e-12)
            cos_mc = (g_mix_normalized @ g_value_normalized).item()
            epoch_cos_mc.append(cos_mc)
            
            # Log norms
            epoch_g_struct_norm.append(g_struct_norm_val)
            epoch_g_value_norm.append(g_value_norm_val)
            epoch_g_mix_norm.append(g_mix_norm_val)
            epoch_loss.append(loss_struct.item())
            
            # Apply mixed gradient
            optimizer.zero_grad()
            idx = 0
            for p in model.parameters():
                numel = p.numel()
                p.grad = g_mix[idx:idx+numel].view(p.shape).clone()
                idx += numel
            optimizer.step()
            global_step += 1
        
        scheduler.step()
        
        # Epoch summary
        acc = evaluate(model, test_loader)
        
        detailed_history['epoch'].append(epoch)
        detailed_history['test_error'].append(1 - acc)
        detailed_history['train_loss'].append(np.mean(epoch_loss))
        
        detailed_history['cos_struct_value_mean'].append(np.mean(epoch_cos_sv))
        detailed_history['cos_struct_value_std'].append(np.std(epoch_cos_sv))
        detailed_history['cos_struct_value_min'].append(np.min(epoch_cos_sv))
        detailed_history['cos_struct_value_max'].append(np.max(epoch_cos_sv))
        
        detailed_history['g_struct_norm_mean'].append(np.mean(epoch_g_struct_norm))
        detailed_history['g_value_norm_mean'].append(np.mean(epoch_g_value_norm))
        detailed_history['g_mix_norm_mean'].append(np.mean(epoch_g_mix_norm))
        
        detailed_history['cos_mix_clean_mean'].append(np.mean(epoch_cos_mc))
        
        if (epoch + 1) % 20 == 0:
            print(f'    Epoch {epoch+1}: error={1-acc:.4f}, cos_sv={np.mean(epoch_cos_sv):.4f}')
    
    final_acc = evaluate(model, test_loader)
    
    return final_acc, detailed_history

In [ ]:
# ===== データ準備 =====
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader, test_loader = get_data_loaders(trainset, testset)

print('Data prepared')

# GPU warmup
warmup_model = get_resnet18().to(device)
for _ in range(20):
    _ = warmup_model(torch.randn(BATCH_SIZE, 3, 32, 32, device=device))
del warmup_model
torch.cuda.empty_cache()
print('GPU warmed up')

In [ ]:
# ===== メイン実験ループ =====
results = []
checkpoint_file = f'{SAVE_DIR}/{EXP_NAME}_checkpoint.json'
completed = set()

if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        results = json.load(f)
    for r in results:
        completed.add((r['noise_rate'], r['lambda'], r['seed']))
    print(f'Checkpoint loaded: {len(results)} runs completed')

total = len(experiments)

for exp in experiments:
    eta = exp['noise_rate']
    lam = exp['lambda']
    phase_label = exp['phase_label']
    seed = exp['seed']
    
    if (eta, lam, seed) in completed:
        continue
    
    run_num = len(completed) + 1
    print(f'\n[{run_num}/{total}] η={eta} λ={lam} ({phase_label}) seed={seed}')
    
    set_seed(seed)
    noisy_labels = inject_label_noise(clean_labels, eta, seed)
    
    model = get_resnet18().to(device)
    
    start_time = time.time()
    final_acc, detailed_history = train_dual_gradient_detailed(
        model, train_loader, test_loader, clean_labels, noisy_labels, lam
    )
    elapsed = time.time() - start_time
    
    result = {
        'experiment_id': f'Q4-{run_num:03d}',
        'experiment': EXP_NAME,
        'noise_rate': eta,
        'lambda': lam,
        'phase_label': phase_label,
        'seed': seed,
        'final_acc': final_acc,
        'final_error': 1 - final_acc,
        'time_seconds': elapsed,
        'detailed_history': detailed_history
    }
    results.append(result)
    completed.add((eta, lam, seed))
    
    status = '✅' if final_acc > 0.85 else ('⚠️ COLLAPSE' if final_acc < 0.5 else '')
    print(f'  Final error: {1-final_acc:.4f} | Time: {elapsed/60:.1f} min {status}')
    
    with open(checkpoint_file, 'w') as f:
        json.dump(results, f, indent=2)
    
    remaining = total - run_num
    eta_hours = remaining * elapsed / 3600
    print(f'  Progress: {run_num}/{total} | ETA: {eta_hours:.1f} hours')
    
    del model
    torch.cuda.empty_cache()

print('\n' + '='*70)
print('ALL EXPERIMENTS COMPLETED!')
print('='*70)

In [ ]:
# ===== 結果保存 =====
import pandas as pd

with open(f'{SAVE_DIR}/{EXP_NAME}_results.json', 'w') as f:
    json.dump(results, f, indent=2)

results_flat = [{k: v for k, v in r.items() if k != 'detailed_history'} for r in results]
df = pd.DataFrame(results_flat)
df.to_csv(f'{SAVE_DIR}/{EXP_NAME}_results.csv', index=False)

print(f'Results saved to {SAVE_DIR}')

In [ ]:
# ===== cos時間発展の可視化 =====
import matplotlib.pyplot as plt

print('='*70)
print('COSINE TEMPORAL EVOLUTION ANALYSIS')
print('='*70)

phase_colors = {'ordered': '#2ecc71', 'critical': '#f39c12', 'collapse': '#e74c3c'}

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

for i, eta in enumerate([0.4, 0.8]):
    # Filter results for this eta
    eta_results = [r for r in results if r['noise_rate'] == eta]
    
    # (a) cos_struct_value over time
    ax = axes[i, 0]
    for phase_label in ['ordered', 'critical', 'collapse']:
        phase_runs = [r for r in eta_results if r['phase_label'] == phase_label]
        if phase_runs:
            # Average over seeds
            all_cos = np.array([r['detailed_history']['cos_struct_value_mean'] for r in phase_runs])
            mean_cos = np.mean(all_cos, axis=0)
            std_cos = np.std(all_cos, axis=0)
            epochs = phase_runs[0]['detailed_history']['epoch']
            
            ax.plot(epochs, mean_cos, color=phase_colors[phase_label], 
                   label=f'{phase_label} (λ={phase_runs[0]["lambda"]})', linewidth=2)
            ax.fill_between(epochs, mean_cos - std_cos, mean_cos + std_cos,
                           color=phase_colors[phase_label], alpha=0.2)
    
    ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel('cos(g_struct, g_value)', fontsize=12)
    ax.set_title(f'η={eta}: Gradient Alignment Over Time', fontsize=12)
    ax.legend(loc='lower right')
    ax.grid(True, alpha=0.3)
    
    # (b) test_error over time
    ax = axes[i, 1]
    for phase_label in ['ordered', 'critical', 'collapse']:
        phase_runs = [r for r in eta_results if r['phase_label'] == phase_label]
        if phase_runs:
            all_error = np.array([r['detailed_history']['test_error'] for r in phase_runs])
            mean_error = np.mean(all_error, axis=0)
            std_error = np.std(all_error, axis=0)
            epochs = phase_runs[0]['detailed_history']['epoch']
            
            ax.plot(epochs, mean_error, color=phase_colors[phase_label],
                   label=f'{phase_label}', linewidth=2)
            ax.fill_between(epochs, mean_error - std_error, mean_error + std_error,
                           color=phase_colors[phase_label], alpha=0.2)
    
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel('Test Error', fontsize=12)
    ax.set_title(f'η={eta}: Learning Curves by Phase', fontsize=12)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # (c) cos vs error scatter (all epochs)
    ax = axes[i, 2]
    for phase_label in ['ordered', 'critical', 'collapse']:
        phase_runs = [r for r in eta_results if r['phase_label'] == phase_label]
        if phase_runs:
            for r in phase_runs[:1]:  # 代表1つだけ
                cos_vals = r['detailed_history']['cos_struct_value_mean']
                error_vals = r['detailed_history']['test_error']
                ax.scatter(cos_vals, error_vals, c=phase_colors[phase_label],
                          alpha=0.5, s=20, label=phase_label if r == phase_runs[0] else '')
    
    ax.axvline(x=0, color='black', linestyle='--', alpha=0.5)
    ax.set_xlabel('cos(g_struct, g_value)', fontsize=12)
    ax.set_ylabel('Test Error', fontsize=12)
    ax.set_title(f'η={eta}: Error vs Alignment (trajectory)', fontsize=12)
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/figures/{EXP_NAME}_temporal.png', dpi=150, bbox_inches='tight')
plt.show()

# cos符号反転の検出
print('\n--- Cosine Sign Reversal Detection ---')
for eta in [0.4, 0.8]:
    print(f'\nη={eta}:')
    eta_results = [r for r in results if r['noise_rate'] == eta]
    for phase_label in ['ordered', 'critical', 'collapse']:
        phase_runs = [r for r in eta_results if r['phase_label'] == phase_label]
        if phase_runs:
            # 最終エポックのcos平均
            final_cos = np.mean([r['detailed_history']['cos_struct_value_mean'][-1] for r in phase_runs])
            # 符号反転したエポックを探す
            reversal_epochs = []
            for r in phase_runs:
                cos_vals = r['detailed_history']['cos_struct_value_mean']
                for j, c in enumerate(cos_vals[:-1]):
                    if c < 0 and cos_vals[j+1] > 0:
                        reversal_epochs.append(j)
                        break
            
            if reversal_epochs:
                print(f'  {phase_label}: final cos={final_cos:.4f}, reversal at epoch≈{np.mean(reversal_epochs):.0f}')
            else:
                print(f'  {phase_label}: final cos={final_cos:.4f}, no reversal')